# Scraping 'Apple Watch Series 4' from Carousell 

## Web Scraping with Beautifulsoup (Python)

### BeautifulSoup
- __Python Package__ to parse HTML and XML content
- __Object__ to parse data from content (HTML Parser)
    - creates a parse tree for parsed pages that can be used to extract data from HTML
- Functions/Methods to extract data from the content
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/

### Functions/Methods
 - .find_all()
     - return all the items that are using the tags you searched for
     - soup.find_all('a')
         - <class 'bs4.element.ResultSet'>
         - <a class="sister" href=(#"http://example.com/elsie") id="link1">Elsie</a>,
         - <a class="sister" href=(#"http://example.com/lacie") id="link2">Lacie</a>,
         - <a class="sister" href=(#"http://example.com/tillie") id="link3">Tillie</a>
     
 - .find()
     - <class 'bs4.element.Tag'>
     - return only 1 item that you searched for
     - find(id="link3")
         - <a class="link" href=(#"http://example.com/example3") id="link3">This returns just the matching element by ID</a>
 - .select()
     - .select("p > a")
         - <class 'list'>
         - <a class="sister" href=(#"http://example.com/elsie") id="link1">Elsie</a>,
         - <a class="sister" href=(#"http://example.com/lacie")  id="link2">Lacie</a>,
         - <a class="sister" href=(#"http://example.com/tillie") id="link3">Tillie</a>
 
### Steps to scrape a website
1. Identify the structure of the website
    - look out for odd cases that requires additional function to handle
2. Plan the flow on how you want to scrape the website
    - Navigation among the pages
3. Estimate the amount of data you will need to scrape
    - As the data increase, the time to run the program will be longer
    - likely to send more request to the webpage server 
4. 'Inspect element' to inspect the source code of the page
    - Understand the HTML code and identify what data you need
5. Writing the code
    - Planning on how to store the data
    - Planning the structure of the code

## 16/07/2019

In [ ]:
#!pip install html5lib


## 17/07/2019

In [15]:
from bs4 import BeautifulSoup
import urllib
import requests
import time
import html5lib
import pandas as pd

print("imports done")

imports done


In [58]:

# Function to send request to page and parse html page into BeautifulSoup Object
def request_page(url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
    req = urllib.request.Request(url, headers=headers)
    html = urllib.request.urlopen(req)
    read_html = html.read()
    page = BeautifulSoup(str(read_html,"utf-8"),  "html.parser")
    return page

count_pages = 1
bike_list = [] #sore all the apple watch 

base = 'https://www.sgbikemart.com.sg/'
url = 'https://www.sgbikemart.com.sg/listing/usedbikes/listing/'
extension = '?page=401&'
pindex=401

#?page=401&
#and int(pindex) < 1

while extension != None :
    
    count_items = 0
    bike_page = request_page(url+extension)
    
    all_bike = bike_page.select('div.row > div.col-lg-9 > div.pmd-card') 
    
    print("Number of items: ", len(all_bike), "Page: ", count_pages)
    print("URL: ", url + extension)

    header=None
    
    for index in all_bike:
        bike_dict ={} #store all the info of an apple watch
        
        #print("Looping through the records")
        
        item_tag ='pmd-card pmd-z-depth'
        
        ad_header, ad_price, ad_currency,ad_biketype,ad_registration, description,ad_posted_on,ad_direct_seller,ad_dealer_seller,ad_paid_flag,expired,ad_status,ad_linkdetails,ad_mileage='','','','','','','','','','','','','',''
        
        # Retrieve data from website using HTML tags
             
        header_tag= 'pmd-card-header text-center-xs'
        header = index.find(name='div', class_=header_tag)
        if header != None:
            if header.find('a') != None:
                ad_header = header.get_text().strip()
                #print("***** Title : " + ad_header + " *****")
        
        if ad_header != None:
            #print(ad_header)
            split_header = ad_header.split()
                
        body_tag ='pmd-card-body'
        body_data= index.find(name='div', class_='col-lg-10')
        
        #print(body_data)
        if body_data != None:
            #for body_item in body_data:

            list_price = body_data.find(name='strong', class_='listing-price')
        
            ad_keydata=body_data.find_all(name='div', class_='col-lg-3')
        
            for adkey in ad_keydata:
                #
                text= adkey.get_text().strip()
                
                #print(text)
                
                if text.startswith('SGD'):
                    ad_price=text
                    
                if text.startswith('Reg'):
                    ad_registration=text[len('Reg :'):].strip()
                    
                if text.startswith('Type'):
                    ad_biketype=text[len('Type: '):].strip()
                    
            ad_keydata=body_data.find_all(name='div', class_='col-lg-4')
            
            for adkey in ad_keydata:
                text= adkey.get_text().strip()
                
                #print(text)
                            
                if text.startswith('Posted'):
                    ad_posted_on=text[len('Posted on :'):].strip()
                    #print("POSTED " + ad_posted_on)
                    
                if 'Paid' in text:
                    ad_paid_flag="True"
                
                if 'Direct' in text:
                    ad_direct_seller="True"
                    
                if 'Dealer' in text:
                    ad_dealer_seller="True"
                 
   
            ad_miledata=body_data.find(name='div', class_='col-lg-2')
            #print(ad_miledata)
            if ad_miledata != None :
     
                ad_mileage=ad_miledata.get_text().strip()[:-2]
             
            ad_linkdata = body_data.find(name='a')
            #print ()
            if ad_linkdata != None:
                ad_linkdetails = base+ad_linkdata['href']
            
            desc_data = body_data.find(name='div', class_='col-lg-12')
            if desc_data != None:
                description = desc_data.get_text().strip()
            
            expired = body_data.find(name='div', class_='status_30')
            
            if expired != None:
                ad_status='expired'
            else :
                ad_status='live'
                
            sold = body_data.find(name='div', class_='status_40')
            
            if sold != None:
                ad_status='sold'
                        
            
        # Storing data into dictionary
        if header!=None :
            
            bike_dict['ad_header'] = ad_header
            bike_dict['ad_price'] = ad_price
            bike_dict['ad_registration'] = ad_registration
            bike_dict['ad_mileage'] = ad_mileage
            bike_dict['ad_biketype'] = ad_biketype
            bike_dict['ad_description'] = description
            bike_dict['ad_posted_on'] = ad_posted_on
            bike_dict['ad_paid_flag'] = ad_paid_flag[0:len('Paid Ad')]
            bike_dict['ad_direct_seller'] = ad_direct_seller
            bike_dict['ad_dealer_seller'] = ad_dealer_seller
            bike_dict['ad_status'] = ad_status
            bike_dict['ad_linkdetails'] = ad_linkdetails
            
            i=1
            for splitter in split_header:
                bike_dict['header_'+str(i)]=splitter
                i+=1

        #print(bike_dict )
        #print("\n")
       
        
        # append the dictionary into a list 
        bike_list.append(bike_dict)
        
        #print("Counter " + str(count_items))
        count_items += 1
    
    print("Number of items scraped: ", count_items)
    print("-" * 40)
        
    # Retrieve new extension (next extension is for the Next page)
    extension = bike_page.find('ul', class_='pagination')
    pindex=extension.find('li', class_='active').get_text()
    
    #print("Index" + pindex)
    next_index=int(pindex)+1
    extension = "?page=" + str(next_index) +"&"

    #print(extension)
    count_pages += 1
    time.sleep(5)

print("--Scraping done!--")



IndentationError: expected an indented block (<ipython-input-58-c97440df65e2>, line 121)

## Pandas Library 

### Pandas
 - DataFrame object for data manipulation and analysis.
 - Pandas is built on top of the __NumPy__ package
 
### Functions
 - .head()
     - Display top 5 data
 - .tail(2)
     - Display bottom 2 data
 - .info()
     - essential details about your dataset, such as the number of rows and columns, the number of non-null values, what type of data is in each column, and how much memory your DataFrame is using.
 - .shape()
     - Display (rows, columns)
 - https://www.learndatasci.com/tutorials/python-pandas-tutorial-complete-introduction-for-beginners/
 
 

## Export data (list of dictionary) to CSV

In [56]:
df = pd.DataFrame.from_dict(bike_list)
df.to_csv('/Users/ganesha/Downloads/Antler/Bizoo/sgbikemartdata.csv')